In [26]:
import pandas as pd

# Battery Specifications (Tesla Powerwall 3 x1)
#BATTERY_CAPACITY_KWH = 13.5
#MAX_CHARGE_RATE_KW = 5  # Continuous power rating

# Battery Specifications (Tesla Powerwall 3 x3)
BATTERY_CAPACITY_KWH = 13.5*3
MAX_CHARGE_RATE_KW = 8  # Continuous power rating

MAX_DISCHARGE_RATE_KW = 11  # Continuous power rating
ROUND_TRIP_EFFICIENCY = 0.89  # Estimated round-trip efficiency


class Battery:
    """Represents the battery with its state and actions."""

    def __init__(self, capacity_kwh, max_charge_rate_kw, max_discharge_rate_kw,
                 round_trip_efficiency):
        self.capacity_kwh = capacity_kwh
        self.max_charge_rate_kw = max_charge_rate_kw
        self.max_discharge_rate_kw = max_discharge_rate_kw
        self.round_trip_efficiency = round_trip_efficiency
        self.charge_kwh = 0.0

    def charge(self, amount_kwh):
        """Charges the battery, respecting capacity and rate limits."""
        actual_charge_kwh = min(
            amount_kwh, self.capacity_kwh - self.charge_kwh)
        self.charge_kwh += actual_charge_kwh * self.round_trip_efficiency
        return actual_charge_kwh

    def discharge(self, amount_kwh):
        """Discharges the battery, respecting capacity and rate limits."""
        actual_discharge_kwh = min(amount_kwh, self.charge_kwh)
        self.charge_kwh -= actual_discharge_kwh
        return actual_discharge_kwh


def simulate_battery(data_filepath):
    """
    Simulates a home battery system, charging from 11:30 PM to 4:30 AM and
    selling to the grid from 3 PM to 6:30 PM.

    If there is consumption when the battery is not charging, the battery capacity is used first.
    If there's excess discharge capacity during the selling period, sell it back to the grid.

    Args:
        data_filepath: Path to the CSV file containing the time-series data.

    Returns:
        A pandas DataFrame with simulation results and the total savings/earnings.
    """

    # Load data
    df = pd.read_csv(data_filepath, parse_dates=['interval_start', 'interval_end'])
    df.sort_values(by='interval_start', inplace=True)
    df.reset_index(drop=True, inplace=True)
    #df['time'] = df['interval_start'].dt.strftime('%H:%M')

    # Initialize battery
    battery = Battery(BATTERY_CAPACITY_KWH, MAX_CHARGE_RATE_KW, MAX_DISCHARGE_RATE_KW, ROUND_TRIP_EFFICIENCY)
    total_cost = 0.0
    total_earnings = 0.0

    results = []

    for index, row in df.iterrows():
        consumption_kwh = 0#row['consumption']
        import_rate = row['buy_rate_inc_vat']
        export_rate = row['sell_rate_inc_vat']
        current_time = row['interval_start']
        

        initial_charge = battery.charge_kwh
        charge_amount_kwh = 0
        discharge_amount_kwh = 0

        # --- Battery Logic ---

        # Charging period
        if current_time.time() >= pd.Timestamp("23:30").time() or current_time.time() < pd.Timestamp("04:30").time():
            charge_amount_kwh = battery.charge(MAX_CHARGE_RATE_KW * 0.5)  # Charge at max rate for half hour
            total_cost += charge_amount_kwh * import_rate

        # Selling period
        elif current_time.time() >= pd.Timestamp("15:00").time() and current_time.time() < pd.Timestamp("18:30").time():
            # First, meet consumption from the battery
            discharge_for_consumption_kwh = battery.discharge(min(consumption_kwh, MAX_DISCHARGE_RATE_KW * 0.5))
            consumption_kwh -= discharge_for_consumption_kwh
            
            # Then, sell the remaining discharge capacity to the grid
            remaining_discharge_capacity = MAX_DISCHARGE_RATE_KW * 0.5 - discharge_for_consumption_kwh
            discharge_to_grid_kwh = battery.discharge(remaining_discharge_capacity)

            discharge_amount_kwh = discharge_for_consumption_kwh + discharge_to_grid_kwh
            total_earnings += discharge_to_grid_kwh * export_rate


        # Consume from battery if not charging and not selling
        elif not (current_time.time() >= pd.Timestamp("23:30").time() or current_time.time() < pd.Timestamp("04:30").time()):
            if consumption_kwh > 0:
                discharge_amount_kwh = battery.discharge(min(consumption_kwh, MAX_DISCHARGE_RATE_KW * 0.5))
                consumption_kwh -= discharge_amount_kwh

        # Buy remaining energy from the grid
        if consumption_kwh > 0:
            total_cost += consumption_kwh * import_rate

        results.append({
            'interval_start': row['interval_start'],
            'interval_end': row['interval_end'],
            'initial_battery_charge_kwh': initial_charge,
            'consumption_kwh': row['consumption'],
            'import_rate': import_rate,
            'export_rate': export_rate,
            'charge_amount_kwh': charge_amount_kwh,
            'discharge_amount_kwh': discharge_amount_kwh,
            'final_battery_charge_kwh': battery.charge_kwh,
            'cost_this_interval': consumption_kwh * import_rate + charge_amount_kwh * import_rate,
            'earnings_this_interval': discharge_amount_kwh * export_rate if current_time.time() >= pd.Timestamp("15:00").time() and current_time.time() < pd.Timestamp("18:30").time() else 0 # Only count earnings during the selling period
        })

    results_df = pd.DataFrame(results)
    net_savings_earnings = total_earnings - total_cost

    return results_df, net_savings_earnings

# Example Usage
data_filepath = 'output.csv'  # Replace with your data file
simulation_results, total_savings_earnings = simulate_battery(data_filepath)

In [23]:


total_cost = simulation_results['consumption_kwh']*simulation_results['import_rate']
#print(simulation_results)
print(f"Total Cost with battery: {-total_savings_earnings/100:.2f}")
print(f"Total Cost without battery: {total_cost.sum()/100:.2f}")
# Print the results
# print(simulation_results)
days = (simulation_results['interval_end'].max() -
        simulation_results['interval_start'].min()).days
print(
    f"\nTotal Savings/Earnings from Battery per year: £{(total_cost.sum()+total_savings_earnings)*365/days/100:.2f}")
simulation_results.to_csv('simulation_results.csv', index=False)
# You can further analyze the simulation_results DataFrame to
# understand battery behavior, charge/discharge patterns, etc.

Total Cost with battery: 325.67
Total Cost without battery: 1114.33

Total Savings/Earnings from Battery per year: £1020.78


In [ ]:
df

In [36]:
df= pd.read_csv('batteries.csv')
output_list = []
for index,row in df.iterrows():
    print(index,end = '\r')
    output_list.append(0)
    BATTERY_CAPACITY_KWH = row['Useable capacity (kWh)']
    MAX_CHARGE_RATE_KW = row['Charge']  # Continuous power rating

    MAX_DISCHARGE_RATE_KW = row['Discharge']  # Continuous power rating
    ROUND_TRIP_EFFICIENCY = row['efficiency']  # Estimated round-trip efficiency
    simulation_results, output_list[index] = simulate_battery(data_filepath)
df['Savings per year'] = output_list
days = (simulation_results['interval_end'].max() -
        simulation_results['interval_start'].min()).days
df['Savings per year'] = df['Savings per year']*365/days/100

In [39]:

df['Savings per year'] = output_list
df['Savings per year'] = df['Savings per year']*365/days/100
df.to_csv('batteries_sim.csv',index = False)